In [17]:
#from PS1QueryFunctions import * 
import pandas as pd
import numpy as np
from astropy.table import Table
import requests
from astropy.io import ascii

In [8]:
data = pd.read_csv('starlist.csv')

In [13]:
def ps1search(table="mean",release="dr1",format="csv",columns=None,baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,**kw):
    """Do a general search of the PS1 catalog (possibly without ra/dec/radius)

    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2).  Note this is required!
    """

    data = kw.copy()
    if not data:
        raise ValueError("You must specify some parameters for search")
    checklegal(table,release)
    if format not in ("csv","votable","json"):
        raise ValueError("Bad value for format")
    url = "{baseurl}/{release}/{table}.{format}".format(**locals())
    if columns:
        # check that column values are legal
        # create a dictionary to speed this up
        dcols = {}
        for col in ps1metadata(table,release)['name']:
            dcols[col.lower()] = 1
        badcols = []
        for col in columns:
            if col.lower().strip() not in dcols:
                badcols.append(col)
        if badcols:
            raise ValueError('Some columns not found in table: {}'.format(', '.join(badcols)))
        # two different ways to specify a list of column values in the API
        # data['columns'] = columns
        data['columns'] = '[{}]'.format(','.join(columns))

# either get or post works
#    r = requests.post(url, data=data)
    r = requests.get(url, params=data)

    if verbose:
        print(r.url)
    r.raise_for_status()
    if format == "json":
        return r.json()
    else:
        return r.text


def ps1cone(ra,dec,radius,table="stack",release="dr1",format="csv",columns=None,baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,**kw):
    """Do a cone search of the PS1 catalog

    Parameters
    ----------
    ra (float): (degrees) J2000 Right Ascension
    dec (float): (degrees) J2000 Declination
    radius (float): (degrees) Search radius (<= 0.5 degrees)
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2)
    """

    data = kw.copy()
    data['ra'] = ra
    data['dec'] = dec
    data['radius'] = radius
    return ps1search(table=table,release=release,format=format,columns=columns,
                    baseurl=baseurl, verbose=verbose, **data)

def checklegal(table,release):
    """Checks if this combination of table and release is acceptable

    Raises a VelueError exception if there is problem
    """

    releaselist = ("dr1", "dr2")
    if release not in ("dr1","dr2"):
        raise ValueError("Bad value for release (must be one of {})".format(', '.join(releaselist)))
    if release=="dr1":
        tablelist = ("mean", "stack")
    else:
        tablelist = ("mean", "stack", "detection")
    if table not in tablelist:
        raise ValueError("Bad value for table (for {} must be one of {})".format(release, ", ".join(tablelist)))

In [10]:
data

,ID,RAJ2000,DEJ2000
0,210619926,57.604035,17.246507
1,210620533,57.596678,17.255203
2,210617295,57.401616,17.209163
3,210622369,57.467123,17.280861
4,210623151,57.424375,17.291832
...,...,...,...
192993,210619812,57.754178,17.244910
192994,210620197,57.733897,17.250397
192995,210622523,57.678534,17.283173
192996,210611806,57.484276,17.131824


In [43]:
a = ps1cone(data['RAJ2000'].iloc[0],data['DEJ2000'].iloc[0],radius=3/60**2)
a = ascii.read(a)
a = a.to_pandas()

In [44]:
len(a)

2

In [45]:
a['gPSFMag']

0   -999.0000
1     12.8575
Name: gPSFMag, dtype: float64

In [36]:
df = data.iloc[:100].copy()

In [118]:
data = pd.read_csv('c16_fkg_stars.csv')

In [119]:
data

,ID,RAJ2000,DEJ2000
0,211402652,132.546697,11.681880
1,211403406,132.451403,11.693481
2,211403356,132.486754,11.692479
3,211403975,132.503507,11.702728
4,211405616,132.487379,11.726850
...,...,...,...
5542,212175620,130.916840,23.623967
5543,212177093,130.919222,23.661489
5544,212174613,130.591100,23.597900
5545,212177120,130.724338,23.661963


In [117]:
df

,objName,objAltName1,objAltName2,objAltName3,objID,uniquePspsOBid,ippObjID,surveyID,htmID,zoneID,...,yKronFlux,yKronFluxErr,yKronRad,yexpTime,yExtNSigma,ysky,yskyErr,yzp,yPlateScale,distance
0,PSO J085011.184+114054.845,-999,-999,-999,122011325466068770,1032313000044885,98320391384918,0,16298226255267,12201,...,0.000307,0.000005,2.41677,730.0,-0.238984,6.254810e-08,0.000003,25.038799,0.25,0.000115


In [131]:
good = []
for i in range(len(data)):
    a = ps1cone(data['RAJ2000'].iloc[i],data['DEJ2000'].iloc[i],radius=3/60**2)
    if a != '':
        a = ascii.read(a)

        a = a.to_pandas()
        
        if len(g) > 1:
            a = a.iloc[0]
        else:
            good += [i]
            
        if i ==0:
            df = a.copy()
        else:
            df = df.append(a)

In [143]:
epic = df.copy()
epic['epic'] = data['ID']
epic['k2ra'] = data['RAJ2000']
epic['k2dec'] = data['DEJ2000']

In [144]:
epic = epic.iloc[good]

In [147]:
epic.to_csv('c16_all_stars.csv',index=False)

In [142]:
data

,ID,RAJ2000,DEJ2000
0,211402652,132.546697,11.681880
1,211403406,132.451403,11.693481
2,211403356,132.486754,11.692479
3,211403975,132.503507,11.702728
4,211405616,132.487379,11.726850
...,...,...,...
5542,212175620,130.916840,23.623967
5543,212177093,130.919222,23.661489
5544,212174613,130.591100,23.597900
5545,212177120,130.724338,23.661963


In [108]:
df = df.drop('Unnamed: 0', axis=1)

In [109]:
df.to_csv('c16_stars_PS.csv',index=False)

In [48]:
g

0   -999.0000
1     12.8575
Name: gPSFMag, dtype: float64

In [101]:
a['gPSFMag']

0     14.0517
1   -999.0000
2     12.5983
Name: gPSFMag, dtype: float64

In [99]:
list(a.keys())

['objName',
 'objAltName1',
 'objAltName2',
 'objAltName3',
 'objID',
 'uniquePspsOBid',
 'ippObjID',
 'surveyID',
 'htmID',
 'zoneID',
 'tessID',
 'projectionID',
 'skyCellID',
 'randomID',
 'batchID',
 'dvoRegionID',
 'processingVersion',
 'objInfoFlag',
 'qualityFlag',
 'raStack',
 'decStack',
 'raStackErr',
 'decStackErr',
 'raMean',
 'decMean',
 'raMeanErr',
 'decMeanErr',
 'epochMean',
 'posMeanChisq',
 'cx',
 'cy',
 'cz',
 'lambda',
 'beta',
 'l',
 'b',
 'nStackObjectRows',
 'nStackDetections',
 'nDetections',
 'ng',
 'nr',
 'ni',
 'nz',
 'ny',
 'uniquePspsSTid',
 'primaryDetection',
 'bestDetection',
 'gippDetectID',
 'gstackDetectID',
 'gstackImageID',
 'gra',
 'gdec',
 'graErr',
 'gdecErr',
 'gEpoch',
 'gPSFMag',
 'gPSFMagErr',
 'gApMag',
 'gApMagErr',
 'gKronMag',
 'gKronMagErr',
 'ginfoFlag',
 'ginfoFlag2',
 'ginfoFlag3',
 'gnFrames',
 'gxPos',
 'gyPos',
 'gxPosErr',
 'gyPosErr',
 'gpsfMajorFWHM',
 'gpsfMinorFWHM',
 'gpsfTheta',
 'gpsfCore',
 'gpsfLikelihood',
 'gpsfQf',
 '

In [148]:
a = pd.read_csv('c16_all_stars.csv')

In [151]:
len(a.keys())

301

In [153]:
a.keys()[-3]

'epic'

In [154]:
301-3

298